In [ ]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''

%matplotlib inline
%load_ext autoreload
%autoreload
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import numpy.ma as ma
import struct
import os
import xarray as xr
import glob
import datetime 
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns

# ESIO Imports

from esio import EsioData as ed

"...in calculating ice extent, algorithms assume the pole hole is  covered by ice at greater than 15 percent, and in calculating ice area, both simply do not include the area of the pole hole."

https://nsidc.org/sites/nsidc.org/files/files/NSIDC-special-report-19.pdf

https://nsidc.org/data/g02135?qt-data_set_tabs=2#qt-data_set_tabs   4.2.3 Arctic Pole Hole


In [ ]:
# General plotting settings
sns.set_style('whitegrid')
sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 2.5})

In [ ]:
#############################################################
# Load in Data
#############################################################

E = ed.EsioData.load()

# Load in Obs
data_dir = E.data_dir
grid_dir = E.grid_dir
# fig_dir = os.path.join(E.fig_dir, 'obs', 'NSIDC_0081' , 'standard')
da_51 = xr.open_mfdataset(E.obs['NSIDC_0051']['sipn_nc']+'_yearly/*.nc', concat_dim='time', autoclose=True, parallel=True)#,
da_81 = xr.open_mfdataset(E.obs['NSIDC_0081']['sipn_nc']+'_yearly/*.nc', concat_dim='time', autoclose=True, parallel=True)#,
da_79 = xr.open_mfdataset(E.obs['NSIDC_0079']['sipn_nc']+'_yearly/*.nc', concat_dim='time', autoclose=True, parallel=True)#,


ds_ext = xr.open_dataset(os.path.join(E.obs['NSIDC_extent']['sipn_nc'], 'N_seaice_extent_daily_v3.0.nc'))
ds_ext = ds_ext.rename({'datetime':'time'})

In [ ]:
DS=datetime.datetime(2015,3,1)
DE=datetime.datetime(2015,4,1)
plt.figure(figsize=(18,6))
ds_ext.Extent.sel(time=slice(DS,DE)).plot(label='Offical', linewidth=4, linestyle='--')
da_51.extent.sel(time=slice(DS,DE)).plot(label='Nasa')
da_81.extent.sel(time=slice(DS,DE)).plot(label='Realtime')
da_79.extent.sel(time=slice(DS,DE)).plot(label='Bootstrap', linestyle='--')
plt.legend()

In [ ]:
plt.figure(figsize=(18,6))
(da_51.extent-ds_ext.Extent).plot(label='Nasa - Offical')
(da_81.extent-ds_ext.Extent).plot(label='Realtime- Offical')
(da_79.extent-ds_ext.Extent).plot(label='Bootstrap- Offical', linestyle='--')
plt.legend()

In [ ]:
(da_81.extent-da_51.extent).plot(label='NRT - NASA TEAM')
np.sqrt(((da_81.extent-da_51.extent)**2).mean()).values